In [1]:
import os
import argparse

import torch
#from tokenization_hanbert import HanBertTokenizer
import pandas as pd
import numpy as np
import collections
import h5py
#from dataloader import KoreanDataset, KoreanBatchGenerator, get_dataloader
from tqdm.notebook import tqdm


In [2]:
from bart.tokenization_hanbert import HanBertTokenizer
tokenizer = HanBertTokenizer.from_pretrained("bart/HanBart-54kN")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'HanBertTokenizer'.


In [3]:
test = """시대가 변함에 따라 아동 폭력의 양상도 바뀌고 있다. 어른에 의해 발생하는 폭력뿐 아니라 또래관계에서 발생하는 폭력도 증가하고 있다. 특히, 인터넷 매체를 통한 언어적 괴롭힘 등 사이버 공간이 새로운 피해현장으로 등장했다. 피해현장에 놓여있는 아이들이 이런 사회변화의 심각성을 적극적으로 알리고 있다. \u200b아동옹호대표기관 초록우산어린이재단(회장 이제훈)은 국제어린이재단연맹(ChildFund Alliance) 국가와 함께 아동폭력을 근절하기 위한 지속적인 노력을 이어나가고 있다. 대한민국에서는 초록우산어린이재단 서울아동옹호센터에서 2018년부터 아동폭력 옹호사업(Child Friendly Accountability, 이하 아동폭력 옹호사업)을 진행하며 아동이 스스로 아동폭력의 심각성을 알려 변화를 촉구하는 옹호활동을 이어나가고 있다. \u200b이 활동의 일환으로, 지난 7월 10일(뉴욕 현지시간) 국제어린이재단연맹 중 아동폭력 옹호사업을 진행하는 4개의 국가(엘살바도르, 우간다, 파라과이, 대한민국)의 대표아동들이 UN을 방문했다. 해당 아동들은 UN HLPF(High Level Political Forum, 이하 고위급정치회담)에 참여하여 아동폭력의 심각성과 국제사회의 관심 및 해결을 촉구하는 시간을 가졌다. \u200b대한민국 대표로 참석한 초록우산어린이재단 이윤서 아동(신수중 3)은 '우리나라에서는 아동폭력이 다른 나라와는 조금 다른 모습으로 나타나고 있다'며 세 가지 아동폭력 상황에 대해 설명했다. \u200b대한민국에서 발생하는 아동폭력 상황은 먼저, 어른들이 교육, 훈육의 모습으로 아동에게 가하는 체벌에 대한 내용이다. 최근 정부에서 발표한 '포용국가 아동정책' 중에는 가정 내 체벌을 없애기 위해 민법상 친권자의 징계권을 개정하겠다는 내용이 포함됐다. 포럼에 참여한 이윤서 아동은 정부의 정책과 연결하여 '아이를 부모의 소유물이라고 생각하는 우리나라 사회 분위기가 먼저 바뀌어야 한다'며 인식개선의 필요성에 대한 의견을 전했다. \u200b두 번째로 또래관계에서 발생하는 신체, 언어, 사이버폭력 심각하게는 성폭력을 예로 들었다. '학교 내에서는 또래관계에서 발생하는 폭력을 해결하기 위한 안전장치가 마련되어 있지만 실효성이 없다'며 대책 마련의 필요성을 공유했다. \u200b마지막으로 지나친 학업과 좋은 성적을 중시하는 사회적 분위기를 꼽았다. '과도한 학업 경쟁으로 인해 한 학원에서는 부모의 동의를 구하고 체벌을 감행하는 경우도 있다'며 '치열해지는 경쟁 속에 친구를 경쟁자로 인식하게 만드는 사회 분위기가 변해야 한다'는 의견을 전했다. \u200b이외에도 본 포럼에 함께 참여한 각 국의 아동대표들은 적극적인 목소리로 아동노동, 조혼, 인신매매 등 아동폭력 심각성을 알렸다. \u200bUN 고위급정치회담(HLPF)은 2016년 9월, UN이 수립한 SDGs(Sustainable Development Goals, 이하 지속가능개발목표) 의제를 점검하는 회의로, 매년 7월 뉴욕에 위치한 UN 본부에서 진행된다. 특히 2019년은 '아동폭력 근절을 위한 전 지구적 협력 방안 모색(Building Momentum: working together for all children to live free from violence)'이라는 주제로 각국 정부 관계자, UN관계자, 학계, 시민사회 인사 등이 참석하여 아동들의 목소리가 더욱 의미 있게 전달될 수 있었다. \u200bUN에서 진행된 고위급정치회담(HLPF) 일정을 함께 한 초록우산어린이재단 국제협력개발1본부 이성호본부장은 '올해는 유엔아동권리협약 비준 30주년을 맞이하는 뜻 깊은 해'라며 '우리의 현재이자, 지속 가능한 발전의 초석이 되는 아동이 직접 참여하여 이번 포럼이 더욱 뜻 깊었다'는 소감을 전했다. \u200b초록우산어린이재단은 국제어린이재단연맹(ChildFund Alliance) 회원국과 함께 SDGs 16.2(지속가능개발목표, 아동폭력) 이슈를 해결하기 위해 UN 회원국가, UN 기관 등을 대상으로 지속적인 옹호활동을 전개해 나갈 예정이다."""

In [4]:
test

"시대가 변함에 따라 아동 폭력의 양상도 바뀌고 있다. 어른에 의해 발생하는 폭력뿐 아니라 또래관계에서 발생하는 폭력도 증가하고 있다. 특히, 인터넷 매체를 통한 언어적 괴롭힘 등 사이버 공간이 새로운 피해현장으로 등장했다. 피해현장에 놓여있는 아이들이 이런 사회변화의 심각성을 적극적으로 알리고 있다. \u200b아동옹호대표기관 초록우산어린이재단(회장 이제훈)은 국제어린이재단연맹(ChildFund Alliance) 국가와 함께 아동폭력을 근절하기 위한 지속적인 노력을 이어나가고 있다. 대한민국에서는 초록우산어린이재단 서울아동옹호센터에서 2018년부터 아동폭력 옹호사업(Child Friendly Accountability, 이하 아동폭력 옹호사업)을 진행하며 아동이 스스로 아동폭력의 심각성을 알려 변화를 촉구하는 옹호활동을 이어나가고 있다. \u200b이 활동의 일환으로, 지난 7월 10일(뉴욕 현지시간) 국제어린이재단연맹 중 아동폭력 옹호사업을 진행하는 4개의 국가(엘살바도르, 우간다, 파라과이, 대한민국)의 대표아동들이 UN을 방문했다. 해당 아동들은 UN HLPF(High Level Political Forum, 이하 고위급정치회담)에 참여하여 아동폭력의 심각성과 국제사회의 관심 및 해결을 촉구하는 시간을 가졌다. \u200b대한민국 대표로 참석한 초록우산어린이재단 이윤서 아동(신수중 3)은 '우리나라에서는 아동폭력이 다른 나라와는 조금 다른 모습으로 나타나고 있다'며 세 가지 아동폭력 상황에 대해 설명했다. \u200b대한민국에서 발생하는 아동폭력 상황은 먼저, 어른들이 교육, 훈육의 모습으로 아동에게 가하는 체벌에 대한 내용이다. 최근 정부에서 발표한 '포용국가 아동정책' 중에는 가정 내 체벌을 없애기 위해 민법상 친권자의 징계권을 개정하겠다는 내용이 포함됐다. 포럼에 참여한 이윤서 아동은 정부의 정책과 연결하여 '아이를 부모의 소유물이라고 생각하는 우리나라 사회 분위기가 먼저 바뀌어야 한다'며 인식개선의 필요성에 대한 의견을 전했다. \u200b두 번

In [5]:
a = tokenizer.encode_plus(test,
    padding='max_length', #'max_length'
    max_length=512,
    truncation=True,
    #add_special_tokens=False,
    return_tensors='np')['input_ids'][0]

In [6]:
print(len(a))
print(len(test))

512
1947


In [7]:
tokenizer.decode(a)

"[CLS] 시대가 변함에 따라 아동 폭력의 양상도 바뀌고 있다. 어른에 의해 발생하는 폭력뿐 아니라 또래관계에서 발생하는 폭력도 증가하고 있다. 특히, 인터넷 매체를 통한 언어적 괴롭힘 등 사이버 공간이 새로운 피해현장으로 등장했다. 피해현장에 놓여있는 아이들이 이런 사회변화의 심각성을 적극적으로 알리고 있다. 아동옹호대표기관 초록우산어린이재단 ( 회장 이제훈 ) 은 국제어린이재단연맹 ( childfund alliance ) 국가와 함께 아동폭력을 근절하기 위한 지속적인 노력을 이어나가고 있다. 대한민국에서는 초록우산어린이재단 서울아동옹호센터에서 2018년부터 아동폭력 옹호사업 ( child friendly accountability, 이하 아동폭력 옹호사업 ) 을 진행하며 아동이 스스로 아동폭력의 심각성을 알려 변화를 촉구하는 옹호활동을 이어나가고 있다. 이 활동의 일환으로, 지난 7월 10일 ( 뉴욕 현지시간 ) 국제어린이재단연맹 중 아동폭력 옹호사업을 진행하는 4개의 국가 ( 엘살바도르, 우간다, 파라과이, 대한민국 ) 의 대표아동들이 un을 방문했다. 해당 아동들은 un hlpf ( high level political forum, 이하 고위급정치회담 ) 에 참여하여 아동폭력의 심각성과 국제사회의 관심 및 해결을 촉구하는 시간을 가졌다. 대한민국 대표로 참석한 초록우산어린이재단 이윤서 아동 ( 신수중 3 ) 은'우리나라에서는 아동폭력이 다른 나라와는 조금 다른 모습으로 나타나고 있다'며 세 가지 아동폭력 상황에 대해 설명했다. 대한민국에서 발생하는 아동폭력 상황은 먼저, 어른들이 교육, 훈육의 모습으로 아동에게 가하는 체벌에 대한 내용이다. 최근 정부에서 발표한'포용국가 아동정책'중에는 가정 내 체벌을 없애기 위해 민법상 친권자의 징계권을 개정하겠다는 내용이 포함됐다. 포럼에 참여한 이윤서 아동은 정부의 정책과 연결하여'아이를 부모의 소유물이라고 생각하는 우리나라 사회 분위기가 먼저 바뀌어야 한다'며 인식개선의 필요성에 대한 의견을 전했다. 두 번째로

In [8]:
import h5py
import torch

In [9]:
sharding_path = '/home/jisu/sharding_corpus/'
h5py_path = "/home/jisu/h5py_dataset/"


In [10]:
file_index='010'

In [11]:
f = h5py.File(h5py_path+f'h5py_data_{file_index}.h5py', "r")
data = torch.from_numpy(f['input_ids'][:]).long()
f.close()

# with open(sharding_path+f'bartcorpus_{file_index}.txt', 'r') as f:
#     text = f.readlines()

In [12]:
#print(len(text))
print(len(data))

300800


In [13]:
tokenizer.batch_decode(data[0:11])

['[CLS] 상록활엽수의 숲 속에서 모여 자란다. 뿌리줄기는 짧고 굵으며 길게 땅 속을 뻗고 잎이 드문드문 달리며 잎자루 밑 부분과 함께 비늘조각이 붙는다. 비늘조각은 줄 모양의 바소꼴 또는 바소꼴이고 길이가 4 7mm, 폭이 1mm이며 가장자리가 밋밋하다. 잎자루는 길이가 30 60cm이고 앞면에 홈이 있으며 연한 녹색 또는 볏짚색이다. 잎몸은 달걀 모양이고 길이가 20 40cm이며 2 3회 깃꼴로 갈라진다. 잎조각은 긴 타원 모양이고 길이가 10 20cm이며 끝 부분이 점점 좁아져서 뾰족하고 밑 부분에 있는 첫째 잎조각이 가장 크다. 작은 잎조각은 긴 타원형의 달걀 모양 또는 좁은 달걀 모양이고 자루가 없으며 털이 없고 표면에 광택이 있으며 가장자리에 톱니가 있고 끝 부분이 까끄라기처럼 뾰족하다. 첫째 잎조각의 뒤쪽 첫째 작은 잎조각이 특히 발달하여 다른 잎조각과 크기가 비슷하고 깃꼴로 갈라진다. 포자낭군은 잎몸 윗부분에 있는 작은 잎조각의 중앙맥과 가장자리 중간에 달려 2줄로 배열하고, 포막은 둥근 신장 모양이며 지름이 1mm 정도이고 가장자리가 밋밋하다. 한국 · 일본 · 타이완 · 중국 · 인도 · 말레이시아 · 폴리네시아 등지에 분포한다. 비슷한 종류인 좀쇠고사리는 뿌리줄기가 짧고 잎이 다닥다닥 달리며 잎몸 끝에 달린 작은잎이 발달하지 않았으며 제주도에서 자란다. [EOS] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [ ]:
# data = """국내 제약업계는 매출액의 95%이상을 내수에 의존하는 전형적인 내수 중심의 산업으로 산업의 특성상 기술집약적 연구개발이 그성장의 바탕이며 거액의 판매비용을 수반하는 일반및 전문의약품 중심
# 으로 시장이 형성되어있다. 의약품은 국민보건과 직결되어 있어 제품개발로부터 출시되기까지 수많은 시험 및 검사과정을 거쳐야 하고 만성적인 공급초과 현상이 빚어져 제약산업은 타 산업에 비해 
# 운전자금 부담이 큰편이다. 제약산업은 gnp 성장에 따른 보건복지비용의 지출확대로 그 시장규모가 꾸준히 신장될 것이나 시장개방, 물질특허제도, 신약재심사제도등에 따른 복제약품 개발한계로 자
# 체 기술력이 취약한 업체는 경쟁력 약화가 예상된다. 또한 의약분업, 완 제수입의약품의 보험약가등재, 보험의약품의 실거래가 상환제, 판매자가격표시제(rpls), 의약외품의 약국외 판매, 포괄수가제
# , 물류조합설립,의약품유통종합정보시스템 등의 급격한 환경변화에 따라 항생제를 중심으로한 약품수요의 감소, 제약업소 및 도매상의 감소, 신약개발의 가속화, 가격경쟁에서 제품경쟁으로의 전환 
# 등이 예상되고 있다. 근래에 각 업체들은 무리한 외형성장을 지양하고 내실경영에 주력하면서 장기적 성장의 바탕인 연구개발투자를 확대하고 있어 의약품의 품질 향상과함께 정부 및 민간업체 주도
# 의 신물질 개발의 연구가 상당부분 진척되고 전국민 건강보험제도의 확대정착으로 지속적인 성장이 기대된다. 치료제 중심의 병원용 의약품은 경기변동의 영향이 크지 않으나, 일반 대중의약품은 경
# 기변동과 계절적 요인에 따른 영향을 비교적 많이 받는다. (4) 경쟁요소자본의 영세성으로 인하여 시장진입이 비교적 용이한 완전 경쟁 시장이므로 의약품 시장의 전면 개방으로 기존 합작사들의 경
# 영권 변동및 영업 강화가 예상되며 자금력을 앞세운 다국적 제약업체의 신규참여가 가속화되고 있고 국내의 대기업들도 제약산업에 속속 참여 하고있어 향후 경쟁이 치열해질것으로 예상된다."""

In [ ]:
a = tokenizer.batch_encode_plus(text[0:10],
    padding='max_length', #'max_length'
    max_length=512,
    truncation=True,
    #add_special_tokens=False,
    return_tensors='pt')['input_ids']

In [27]:
sum(sum(data[0:10] == a))

tensor(5120)

In [56]:
text[-12]

'시골 노인들의 음독자살을 막겠다고 도내 농촌지역 곳곳에 설치된 농약빈병수거함이 설치 이후 수거를 제때 안 하며 잔류농약이 담긴 병들로 잔뜩 쌓인 채 방치된 것으로 드러났다.\n'

In [14]:
import os
import random 



In [15]:
file_list = os.listdir('h5py_dataset')


In [16]:
file_list

['h5py_data_108.h5py',
 'h5py_data_001.h5py',
 'h5py_data_133.h5py',
 'h5py_data_204.h5py',
 'h5py_data_063.h5py',
 'h5py_data_202.h5py',
 'h5py_data_085.h5py',
 'h5py_data_072.h5py',
 'h5py_data_006.h5py',
 'h5py_data_091.h5py',
 'h5py_data_005.h5py',
 'h5py_data_047.h5py',
 'h5py_data_020.h5py',
 'h5py_data_103.h5py',
 'h5py_data_254.h5py',
 'h5py_data_058.h5py',
 'h5py_data_140.h5py',
 'h5py_data_009.h5py',
 'h5py_data_175.h5py',
 'h5py_data_167.h5py',
 'h5py_data_071.h5py',
 'h5py_data_208.h5py',
 'h5py_data_087.h5py',
 'h5py_data_214.h5py',
 'h5py_data_080.h5py',
 'h5py_data_043.h5py',
 'h5py_data_225.h5py',
 'h5py_data_102.h5py',
 'h5py_data_240.h5py',
 'h5py_data_192.h5py',
 'h5py_data_158.h5py',
 'h5py_data_155.h5py',
 'h5py_data_188.h5py',
 'h5py_data_138.h5py',
 'h5py_data_222.h5py',
 'h5py_data_060.h5py',
 'h5py_data_088.h5py',
 'h5py_data_048.h5py',
 'h5py_data_216.h5py',
 'h5py_data_041.h5py',
 'h5py_data_013.h5py',
 'h5py_data_203.h5py',
 'h5py_data_036.h5py',
 'h5py_data

In [17]:
random.shuffle(file_list)

In [19]:
from bart.dataloader import h5pyDataset, h5pyBatchGenerator, get_dataloader


ModuleNotFoundError: No module named 'noise_function'